# Sistemas inteligentes para respostas a perguntas médicas

Gyovana M. Moriyama (216190)

Rafael A. Matumoto (273085)

In [8]:
!pip install -qU langchain_openai langchain-community faiss-cpu sentence-transformers openai datasets pydantic langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 1.9 MB/s eta 0:00:00


In [9]:
import os
import datasets
import datetime
import re
import pandas as pd
import numpy as np
from pprint import pprint
from google.colab import userdata, drive

from tqdm import tqdm
from pydantic import BaseModel, Field
from typing import List, Optional, Literal

from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceBgeEmbeddings

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools.retriever import create_retriever_tool
from langchain_core.load import dumps, loads

In [10]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
os.environ['OPENAI_API_KEY'] = userdata.get('api_key')

In [12]:
# folder_path = '/content/drive/MyDrive/ia024a/projeto/entrega4/'
folder_path = '/content/drive/MyDrive/IA024A_Processamento_de_Linguagem_Natural/Projeto Final/projeto/entrega4/'

## Analysis of answers from each system

### Load results from each system

In [7]:
files = [
    'direct.csv',
    'cot.csv',
    'react.csv'
]

In [14]:
pd.read_csv(folder_path + 'results/' +  'direct.csv', index_col=0)

,question,answer,options,meta_info,answer_idx,metamap_phrases,predictions
0,A junior orthopaedic surgery resident is compl...,Tell the attending that he cannot fail to disc...,{'A': 'Disclose the error to the patient and p...,step1,B,['junior orthopaedic surgery resident' 'comple...,A
1,A 67-year-old man with transitional cell carci...,Cross-linking of DNA,"{'A': 'Inhibition of proteasome', 'B': 'Hypers...",step1,D,['67 year old man' 'transitional cell carcinom...,C
2,Two weeks after undergoing an emergency cardia...,Cholesterol embolization,"{'A': 'Renal papillary necrosis', 'B': 'Choles...",step2&3,B,['Two weeks' 'emergency cardiac' 'stenting' 'u...,B
3,A 39-year-old woman is brought to the emergenc...,"Lactose-fermenting, gram-negative rods forming...","{'A': 'Coagulase-positive, gram-positive cocci...",step1,D,['year old woman' 'brought' 'emergency departm...,B
4,A 35-year-old man comes to the physician becau...,Ketotifen eye drops,"{'A': 'Erythromycin ointment', 'B': 'Ketotifen...",step2&3,B,['35 year old man' 'physician' 'of itchy' 'wat...,B
...,...,...,...,...,...,...,...
1268,A 39-year-old woman presents to the clinic for...,Thyroid-stimulating hormone (TSH),"{'A': 'Glucose', 'B': 'Triiodothyronine (T3)',...",step2&3,D,['year old woman presents' 'clinic' 'annual ph...,D
1269,A 38-year-old woman comes to the physician bec...,Medication abuse,"{'A': 'Celiac disease', 'B': 'Carcinoid tumor'...",step2&3,D,['year old woman' 'physician' 'persistent diar...,A
1270,A 21-year-old college student comes to the phy...,Stop evening exercise,"{'A': 'Trial of diphenhydramine', 'B': 'Stop e...",step2&3,B,['21-year-old college student' 'physician' 'of...,D
1271,A 19-year-old man is brought to the physician ...,Schizotypal personality disorder,"{'A': 'Social anxiety disorder', 'B': 'Avoidan...",step2&3,C,['year old man' 'brought' 'physician' 'mother'...,C


In [15]:
pd.read_csv(folder_path + 'results/' +  'cot.csv', index_col=0)

,question,answer,options,meta_info,answer_idx,metamap_phrases,predictions,reasoning
0,A junior orthopaedic surgery resident is compl...,Tell the attending that he cannot fail to disc...,{'A': 'Disclose the error to the patient and p...,step1,B,['junior orthopaedic surgery resident' 'comple...,B,"', step_by_step='1. The resident is faced with..."
1,A 67-year-old man with transitional cell carci...,Cross-linking of DNA,"{'A': 'Inhibition of proteasome', 'B': 'Hypers...",step1,D,['67 year old man' 'transitional cell carcinom...,B,"', step_by_step='1. The patient has transition..."
2,Two weeks after undergoing an emergency cardia...,Cholesterol embolization,"{'A': 'Renal papillary necrosis', 'B': 'Choles...",step2&3,B,['Two weeks' 'emergency cardiac' 'stenting' 'u...,B,"', step_by_step=\""1. The patient has a history..."
3,A 39-year-old woman is brought to the emergenc...,"Lactose-fermenting, gram-negative rods forming...","{'A': 'Coagulase-positive, gram-positive cocci...",step1,D,['year old woman' 'brought' 'emergency departm...,B,"', step_by_step=\""The patient presents with fe..."
4,A 35-year-old man comes to the physician becau...,Ketotifen eye drops,"{'A': 'Erythromycin ointment', 'B': 'Ketotifen...",step2&3,B,['35 year old man' 'physician' 'of itchy' 'wat...,B,"', step_by_step='The patient is experiencing i..."
...,...,...,...,...,...,...,...,...
1268,A 39-year-old woman presents to the clinic for...,Thyroid-stimulating hormone (TSH),"{'A': 'Glucose', 'B': 'Triiodothyronine (T3)',...",step2&3,D,['year old woman presents' 'clinic' 'annual ph...,D,"', step_by_step='1. The patient is a 39-year-o..."
1269,A 38-year-old woman comes to the physician bec...,Medication abuse,"{'A': 'Celiac disease', 'B': 'Carcinoid tumor'...",step2&3,D,['year old woman' 'physician' 'persistent diar...,A,"', step_by_step=\""1. The patient is a 38-year-..."
1270,A 21-year-old college student comes to the phy...,Stop evening exercise,"{'A': 'Trial of diphenhydramine', 'B': 'Stop e...",step2&3,B,['21-year-old college student' 'physician' 'of...,D,"', step_by_step='The patient presents with sym..."
1271,A 19-year-old man is brought to the physician ...,Schizotypal personality disorder,"{'A': 'Social anxiety disorder', 'B': 'Avoidan...",step2&3,C,['year old man' 'brought' 'physician' 'mother'...,C,"', step_by_step='1. The patient exhibits signi..."


In [16]:
df = pd.DataFrame()
for f in files:
    if len(df) == 0:
        df = pd.read_csv(folder_path + 'results/' + f, index_col=0)
        col = f.split('.')[0]
        df[col] = df['predictions']
        df.drop(columns=['predictions'], inplace=True)
    else:
        tmp = pd.read_csv(folder_path + 'results/' + f, index_col=0)
        col = f.split('.')[0]
        df[col] = tmp['predictions']
        df[col + '_reasoning'] = tmp['reasoning']
df

,question,answer,options,meta_info,answer_idx,metamap_phrases,direct,cot,cot_reasoning,react,react_reasoning
0,A junior orthopaedic surgery resident is compl...,Tell the attending that he cannot fail to disc...,{'A': 'Disclose the error to the patient and p...,step1,B,['junior orthopaedic surgery resident' 'comple...,A,B,"', step_by_step='1. The resident is faced with...",A,"To solve this problem, I need to analyze the e..."
1,A 67-year-old man with transitional cell carci...,Cross-linking of DNA,"{'A': 'Inhibition of proteasome', 'B': 'Hypers...",step1,D,['67 year old man' 'transitional cell carcinom...,C,B,"', step_by_step='1. The patient has transition...",D,"To answer this question, I need to identify th..."
2,Two weeks after undergoing an emergency cardia...,Cholesterol embolization,"{'A': 'Renal papillary necrosis', 'B': 'Choles...",step2&3,B,['Two weeks' 'emergency cardiac' 'stenting' 'u...,B,B,"', step_by_step=\""1. The patient has a history...",B,"To solve this question, I need to analyze the ..."
3,A 39-year-old woman is brought to the emergenc...,"Lactose-fermenting, gram-negative rods forming...","{'A': 'Coagulase-positive, gram-positive cocci...",step1,D,['year old woman' 'brought' 'emergency departm...,B,B,"', step_by_step=\""The patient presents with fe...",D,"To answer the question, I need to analyze the ..."
4,A 35-year-old man comes to the physician becau...,Ketotifen eye drops,"{'A': 'Erythromycin ointment', 'B': 'Ketotifen...",step2&3,B,['35 year old man' 'physician' 'of itchy' 'wat...,B,B,"', step_by_step='The patient is experiencing i...",B,"To solve this question, I need to analyze the ..."
...,...,...,...,...,...,...,...,...,...,...,...
1268,A 39-year-old woman presents to the clinic for...,Thyroid-stimulating hormone (TSH),"{'A': 'Glucose', 'B': 'Triiodothyronine (T3)',...",step2&3,D,['year old woman presents' 'clinic' 'annual ph...,D,D,"', step_by_step='1. The patient is a 39-year-o...",D,"To answer the question, I need to analyze the ..."
1269,A 38-year-old woman comes to the physician bec...,Medication abuse,"{'A': 'Celiac disease', 'B': 'Carcinoid tumor'...",step2&3,D,['year old woman' 'physician' 'persistent diar...,A,A,"', step_by_step=\""1. The patient is a 38-year-...",unanswered,To determine the most likely underlying cause ...
1270,A 21-year-old college student comes to the phy...,Stop evening exercise,"{'A': 'Trial of diphenhydramine', 'B': 'Stop e...",step2&3,B,['21-year-old college student' 'physician' 'of...,D,D,"', step_by_step='The patient presents with sym...",D,To determine the most appropriate recommendati...
1271,A 19-year-old man is brought to the physician ...,Schizotypal personality disorder,"{'A': 'Social anxiety disorder', 'B': 'Avoidan...",step2&3,C,['year old man' 'brought' 'physician' 'mother'...,C,C,"', step_by_step='1. The patient exhibits signi...",C,To determine the most likely diagnosis for thi...


## CoT vs ReAct

In [18]:
# Create a cross-tabulation of 'cot' and 'react' columns
cross_tab = pd.crosstab(df['cot'], df['react'])
cross_tab

react,A,B,C,D,unanswered
cot,,,,,
A,282,24,20,28,12
B,22,249,24,17,16
C,22,10,276,22,12
D,14,12,15,192,4


In [19]:
diff = df['cot'] != df['react']
# Count the number of times the columns are different
different_count = diff.sum()

print(f"The 'cot' and 'react' columns have different values {different_count} times.")

The 'cot' and 'react' columns have different values 274 times.


In [20]:
# Get the sum of diagonal elements (where cot and react are equal)
equal_count = cross_tab.values.diagonal().sum()

print(f"The number of times 'cot' and 'react' columns are equal: {equal_count}")

The number of times 'cot' and 'react' columns are equal: 999


In [29]:
#CoT
print('Number of hits: ', df[df['cot'] == df['answer_idx']].shape[0])
print('Number of errors: ', df[df['cot'] != df['answer_idx']].shape[0])

Number of hits:  940
Number of errors:  333


In [33]:
#React
print('Number of hits: ', df[df['react'] == df['answer_idx']].shape[0])
print('Number of errors: ', df[df['react'] != df['answer_idx']].shape[0])

Number of hits:  951
Number of errors:  322


In [32]:
#Both
print('Number of hits: ', df[(df['cot'] == df['answer_idx']) & (df['react'] == df['answer_idx'])].shape[0])
print('Number of errors: ', df[(df['cot'] != df['answer_idx']) & (df['react'] != df['answer_idx'])].shape[0])

Number of hits:  834
Number of errors:  216


## Wrong answers

Analysis of answers that both systems got wrong.

In [34]:
err =  df[(df['cot'] != df['answer_idx']) & (df['react'] != df['answer_idx'])]
err

,question,answer,options,meta_info,answer_idx,metamap_phrases,direct,cot,cot_reasoning,react,react_reasoning
5,A 39-year-old man presents to the emergency de...,Reassurance and continuous monitoring,"{'A': 'Diltiazem', 'B': 'Labetalol', 'C': 'Pro...",step2&3,D,['year old man presents' 'emergency department...,B,B,"', step_by_step=\""1. The patient is a 39-year-...",B,To determine the first step in management for ...
6,A 68-year-old male comes to the physician for ...,Common iliac artery aneurysm,"{'A': 'Renal artery stenosis', 'B': 'Benign pr...",step1,C,['68 year old male' 'physician' 'evaluation' '...,B,B,"', step_by_step=\""1. The patient is a 68-year-...",B,To determine the most likely underlying cause ...
19,A 46-year-old man is brought to the emergency ...,"Benzodiazepine intoxication\n""","{'A': 'Hypoglycemia', 'B': 'Ethanol intoxicati...",step2&3,D,['year old man' 'brought' 'emergency departmen...,B,B,"', step_by_step=\""1. The patient is a 46-year-...",B,To determine the most likely cause of this pat...
23,A 62-year-old patient has been hospitalized fo...,Staphylococcus aureus,"{'A': 'Streptococcus pneumoniae', 'B': 'Mycoba...",step1,D,['62 year old patient' 'hospitalized' 'week' '...,A,A,"', step_by_step=\""1. The patient is a 62-year-...",A,To determine the most likely causal microorgan...
31,A 72-year-old woman is admitted to the intensi...,Increased ventricular wall stiffness,"{'A': 'Mitral valve regurgitation', 'B': 'Incr...",step1,C,['72 year old woman' 'admitted to' 'intensive ...,D,D,"', step_by_step=\""1. The patient is a 72-year-...",D,"To answer the question, I need to analyze the ..."
...,...,...,...,...,...,...,...,...,...,...,...
1250,A 37-year-old woman with an HIV infection come...,Efavirenz,"{'A': 'Efavirenz', 'B': 'Ritonavir', 'C': 'Ral...",step1,A,['year old woman' 'HIV infection' 'physician' ...,C,C,"', step_by_step='1. The patient is experiencin...",C,"To answer the question, I need to identify the..."
1262,A 30-year-old man presents to the emergency ro...,Blocking antibody attack on the O antigen,{'A': 'ADP ribosylation of elongation factor 2...,step1,B,['30 year old man presents' 'emergency room' '...,A,A,"', step_by_step=\""The patient's symptoms, incl...",unanswered,"To solve this question, I need to analyze the ..."
1265,A 74-year-old woman with a history of hyperten...,Delirium,"{'A': 'Alzheimer dementia', 'B': 'Delirium', '...",step1,B,['74 year old woman' 'history of hypertension'...,C,C,"', step_by_step=\""1. The patient is a 74-year-...",C,To determine the most likely diagnosis for thi...
1269,A 38-year-old woman comes to the physician bec...,Medication abuse,"{'A': 'Celiac disease', 'B': 'Carcinoid tumor'...",step2&3,D,['year old woman' 'physician' 'persistent diar...,A,A,"', step_by_step=\""1. The patient is a 38-year-...",unanswered,To determine the most likely underlying cause ...


In [35]:
pd.crosstab(err['cot'], err['react'])

react,A,B,C,D,unanswered
cot,,,,,
A,43,4,1,5,5
B,5,46,2,4,5
C,4,0,39,4,4
D,3,1,2,37,2


In [50]:
eq = err[err['cot'] == err['react']]
eq.shape[0]

165

In [42]:
def print_question(df, row_idx):
    question = df.iloc[row_idx]['question']
    options = df.iloc[row_idx]['options']
    answer = df.iloc[row_idx]['answer_idx']
    cot = df.iloc[row_idx]['cot']
    cot_reasoning = df.iloc[row_idx]['cot_reasoning']
    react = df.iloc[row_idx]['react']
    react_reasoning = df.iloc[row_idx]['react_reasoning']
    print('Question: ', question)
    print('Options: ', options)
    print('Answer: ', answer)
    print('CoT Answer: ', cot)
    print('CoT Reasoning: ', cot_reasoning)
    print('React Answer: ', react)
    print('React Reasoning: ', react_reasoning)

In [45]:
#Question that both got wrong and the same answer
print_question(eq, 5)

Question:  An otherwise healthy 50-year-old man comes to the physician because of a 6-month history of increasingly frequent episodes of upper abdominal pain, nausea, vomiting, and diarrhea. He has had a 3.2-kg (7-lb) weight loss during this time. Physical examination shows bilateral pitting pedal edema. An endoscopy shows prominent rugae in the gastric fundus. Biopsy shows parietal cell atrophy. Which of the following is the most likely underlying cause?
Options:  {'A': 'Serotonin-secreting gastric tumor', 'B': 'Proliferation of gastric mucus-producing cells', 'C': 'Excessive somatostatin secretion', 'D': 'Ectopic secretion of gastrin'}
Answer:  B
CoT Answer:  D
CoT Reasoning:  ', step_by_step=\"1. The patient presents with upper abdominal pain, nausea, vomiting, diarrhea, and weight loss, along with physical examination findings of bilateral pitting edema. \\n2. The endoscopy reveals prominent rugae in the gastric fundus, which suggests gastric mucosal hypertrophy or enlargement. \\n

In [48]:
print_question(eq, 6)

Question:  A 65-year-old woman schedules an appointment with her physician for evaluation of weight loss and weakness. The weakness is more noticeable when climbing stairs and combing hair. The weakness improves after brief exercise. She also has bladder incontinence for the past 4 weeks and was given an anticholinesterase agent with no improvement. What is the most likely cause of the symptoms?
Options:  {'A': 'Duchenne muscular dystrophy', 'B': 'Hypothyroidism', 'C': 'Myasthenia gravis', 'D': 'Paraneoplastic syndrome from small cell carcinoma of the lung'}
Answer:  D
CoT Answer:  C
CoT Reasoning:  ', step_by_step=\"1. The patient is a 65-year-old woman presenting with weight loss, weakness, and bladder incontinence. \\n2. The weakness is more pronounced with activities like climbing stairs and combing hair, which suggests a muscle-related issue. \\n3. The fact that the weakness improves after brief exercise is characteristic of myasthenia gravis, where muscle strength can temporarily

In [51]:
diff = err[err['cot'] != err['react']]

In [57]:
diff.shape[0]

51

In [52]:
diff

,question,answer,options,meta_info,answer_idx,metamap_phrases,direct,cot,cot_reasoning,react,react_reasoning
69,A 65-year-old man with a history of hypertensi...,Proceed with liver biopsy,"{'A': 'Proceed with liver biopsy', 'B': 'Refer...",step1,A,['65-year-old man' 'history of hypertension' '...,D,D,"', step_by_step=\""1. The patient has a history...",unanswered,To determine the next step in management for t...
185,A previously healthy 18-month-old boy is broug...,Seek a court order for neoadjuvant chemotherapy,{'A': 'Help the parents to arrange supportive ...,step2&3,D,['healthy' 'month old boy' 'brought' 'physicia...,C,C,"', step_by_step=\""1. The patient has been diag...",unanswered,"In this scenario, the physician is faced with ..."
191,A stillborn infant is delivered at 38 weeks' g...,Maternal diabetes,"{'A': 'Maternal oligohydramnios', 'B': 'Intrau...",step1,C,['stillborn infant' 'delivered' 'weeks' 'gesta...,B,B,"', step_by_step=\""1. The infant is stillborn a...",A,To determine the strongest predisposing factor...
203,A 39-year-old man presents to his primary care...,Benznidazole,"{'A': 'Benznidazole', 'B': 'Ivermectin', 'C': ...",step1,A,['year old man presents' 'primary care physici...,D,D,"', step_by_step='The patient presents with sev...",B,"To answer the question, I need to analyze the ..."
204,A 10-year-old Caucasian female with Turner's s...,Inferior mesenteric artery,"{'A': 'Superior mesenteric artery', 'B': 'Infe...",step1,B,"['A 10 year old Caucasian female' ""Turner's sy...",A,A,"', step_by_step='During embryological developm...",unanswered,"To answer the question, I need to understand t..."
271,A 22-year-old primigravida presents for a regu...,No tests required,"{'A': 'HbA1c measurement', 'B': 'No tests requ...",step2&3,B,['year old primigravida presents' 'regular pre...,C,C,"', step_by_step=\""1. The patient is a 22-year-...",D,To determine the cause of the abnormal dipstic...
332,A 55-year-old man presents to the emergency de...,Inhibition of P-glycoprotein,"{'A': 'Drug-induced hypokalemia', 'B': 'Increa...",step1,D,['55 year old man presents' 'emergency departm...,A,A,"', step_by_step=\""1. The patient has been on d...",unanswered,"To answer the question, I need to analyze the ..."
348,A 23-year-old woman presents to her psychiatri...,Continue current therapy,"{'A': 'Add lithium to treatment regimen', 'B':...",step2&3,C,['23 year old woman presents' 'psychiatrist co...,B,B,"', step_by_step='1. The patient presents with ...",A,"To answer this question, I need to analyze the..."
355,A 32-year-old G1P1 patient presents to her obs...,"HIV, syphilis, and hepatitis B","{'A': 'HIV, syphilis, and hepatitis B', 'B': '...",step2&3,A,['year old G1P1 patient presents' 'obstetricia...,D,D,"', step_by_step=\""In the case of a pregnant wo...",unanswered,"To answer the question, I need to determine wh..."
378,A 36-year-old man is brought to the emergency ...,Inflammation and edema,"{'A': 'Inflammation and edema', 'B': 'Abnormal...",step2&3,A,['36 year old man' 'brought' 'emergency depart...,C,D,"', step_by_step=\""The patient presents with mu...",C,To determine the underlying cause of this pati...


In [54]:
print_question(diff, 0)

Question:  A 65-year-old man with a history of hypertension, obesity, and alcoholic cirrhosis is seen in clinic for follow-up. He feels well and currently drinks 5 glasses of wine each night. Medications include atenolol and lisinopril. On physical exam, temperature is 98.1 deg F (36.7 deg C), blood pressure is 151/82 mmHg, pulse is 71/min, and respirations are 14/min. He has spider angiomata on his chest; no asterixis, jaundice, ascites, or peripheral edema is noted. Screening ultrasound reveals a new liver nodule, and follow up CT demonstrates a 2 cm right hepatic lobe lesion with enhancement in the arterial phase. No hypodense attenuation is seen on the venous or delayed phase. What is the next step in management?
Options:  {'A': 'Proceed with liver biopsy', 'B': 'Refer for surgical resection', 'C': 'Refer for radiofrequency ablation', 'D': 'Observe and get follow-up imaging in 3 months'}
Answer:  A
CoT Answer:  D
CoT Reasoning:  ', step_by_step=\"1. The patient has a history of alc

In [56]:
print_question(diff, 2)

Question:  A stillborn infant is delivered at 38 weeks' gestation to a 32-year-old woman. The mother had no prenatal care. Examination of the stillborn shows a small pelvis, shallow intergluteal cleft, and club feet. An x-ray shows the absence of the sacrum and lumbar vertebrae. Which of the following is the strongest predisposing factor for this patient's condition?
Options:  {'A': 'Maternal oligohydramnios', 'B': 'Intrauterine alcohol exposure', 'C': 'Maternal diabetes', 'D': 'Chromosome 18 trisomy'}
Answer:  C
CoT Answer:  B
CoT Reasoning:  ', step_by_step=\"1. The infant is stillborn and exhibits several congenital anomalies: a small pelvis, shallow intergluteal cleft, club feet, absence of the sacrum and lumbar vertebrae. \\n2. These findings suggest a condition related to abnormal fetal development, possibly linked to teratogenic exposure. \\n3. Among the options given, intrauterine alcohol exposure is known to cause a range of developmental issues including fetal alcohol syndrom